<a href="https://colab.research.google.com/github/AlexLZM/ML-lab-Final-Project/blob/main/ML_lab_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.base import BaseEstimator
from sklearn.compose import *
from sklearn.ensemble import (
    AdaBoostClassifier,
    GradientBoostingClassifier,
    HistGradientBoostingClassifier,
    RandomForestClassifier,
    StackingClassifier,
    VotingClassifier,
)
from sklearn.experimental import enable_hist_gradient_boosting, enable_iterative_imputer
from sklearn.impute import *
from sklearn.inspection import permutation_importance
from sklearn.metrics import *
from sklearn.model_selection import *
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import *


class DummyEstimator(BaseEstimator):
    def fit(self): pass
    def score(self): pass

# MSDS 699 Final Project

## Student: Zhimin Lyu

## Project description

This dataset gives information of patients related to heart disease. Dataset contains 13 features of patients' conditons and test results, target variable is the fact that the patient has heart disease or not. 

In the project, we set the aim to build a decent model to predict the target variable (disease\non disease) by comparing different machine learning algorithms.

<b>Data Attribute Information</b>

Age (age in years)

Sex (1 = male; 0 = female)

CP (chest pain type)

TRESTBPS (resting blood pressure (in mm Hg on admission to the hospital))

CHOL (serum cholestoral in mg/dl)

FPS (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)

RESTECH (resting electrocardiographic results)

THALACH (maximum heart rate achieved)

EXANG (exercise induced angina (1 = yes; 0 = no))

OLDPEAK (ST depression induced by exercise relative to rest)

SLOPE (the slope of the peak exercise ST segment)

CA (number of major vessels (0-3) colored by flourosopy)

THAL (3 = normal; 6 = fixed defect; 7 = reversable defect)

Disease (has heart disease or not: 1 or 0)

## Load Data

In [ ]:
data = pd.read_csv(
    'https://github.com/AlexLZM/Testit/raw/main/Heart_Disease%20(1).csv')

In [ ]:
# Data split
X, y = data.iloc[:, :-1], data.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y)

## Fit scikit-learn model

<b>Create pipeline for categorical and numeric values</b>

In [ ]:

# seperate categorical and numerical based on domain knowlage
cate = [0, 2, 3, 7, 9, 11, 13]
nume = [1, 4, 5, 6, 8, 9, 12]

con_pipe = Pipeline([('sca', StandardScaler()),  # apply standardscaler to numeric features
                     ('imputer', SimpleImputer(strategy='median'))])  # fill null value with median
cat_pipe = Pipeline([('str',
                      FunctionTransformer(lambda x:(x.astype(str)),
                                          validate=False)),  # transform integer categoricals in to string
                     # fill null value with 'unknown'
                     ('imputer', SimpleImputer(
                         strategy='constant', fill_value='unknown')),
                     ('encoder', OneHotEncoder(handle_unknown='ignore'))])  # apply one-hot encoder to categoricals
prep = ColumnTransformer(
    [('conti', con_pipe, nume), ('cate', cat_pipe, cate)])
# Dummy for Randomized search
pipe = Pipeline([('prep', prep), ('clf', DummyEstimator())])

In [ ]:
# Randomized search withing hyperparameters and model families
search_space = [{'clf': [RandomForestClassifier(n_jobs=-1)],  # Estimator 1
                 # controls dependence of trees
                 'clf__max_features': np.arange(0.1, 0.6, 0.1),
                 # should be enough to converge to minimum validation loss
                 'clf__n_estimators': np.arange(100, 1000, 100),
                 # control tree size to reduce overfitting
                 'clf__min_samples_leaf':np.arange(1, 22, 5),
                 # control tree size to reduce overfitting
                 'clf__max_depth': np.arange(1, 22, 2),
                 # control tree size to reduce overfitting
                 'clf__max_leaf_nodes': np.arange(10, 200, 20),
                 # bootstrap/subsampling choice
                 'clf__bootstrap': [True, False],
                 # control tree size to reduce overfitting
                 'clf__min_samples_split': np.arange(2, 22, 2)
                 },

                {'clf': [GradientBoostingClassifier()],  # Estimator 2
                 # combine with learning rate to reduce overfitting
                 'clf__n_estimators': np.arange(100, 1000, 100),
                 # control tree size to reduce overfitting
                 'clf__min_samples_leaf':np.arange(1, 22, 5),
                 # control tree size to reduce overfitting
                 'clf__max_depth': np.arange(1, 22, 2),
                 # lower lr means slower fitting process
                 'clf__learning_rate':np.arange(0.1, 1.1, 0.1),
                 # control tree size to reduce overfitting
                 'clf__max_leaf_nodes': np.arange(10, 200, 20),
                 # control tree size to reduce overfitting
                 'clf__min_samples_split': np.arange(2, 22, 2)
                 },
                {'clf': [AdaBoostClassifier()],  # Estimator 3
                 # combine with learning rate to reduce overfitting
                 'clf__n_estimators': np.arange(100, 1000, 100),
                 # lower lr means slower fitting process
                 'clf__learning_rate':np.arange(0.1, 1.1, 0.1),
                 },
                {'clf': [HistGradientBoostingClassifier()],  # Estimator 4
                 # control tree size to reduce overfitting
                 'clf__min_samples_leaf':np.arange(1, 22, 5),
                 # control tree size to reduce overfitting
                 'clf__max_depth': np.arange(1, 22, 2),
                 # control tree size to reduce overfitting
                 'clf__max_leaf_nodes': np.arange(10, 200, 20),
                 # lower lr means slower fitting process
                 'clf__learning_rate':np.arange(0.1, 1.1, 0.1),
                 # reduce overfit but increase bias
                 'clf__l2_regularization':np.logspace(0.00001, 10, 10)

                 }
                ]

# random search instance
rand_search = RandomizedSearchCV(estimator=pipe,
                                 param_distributions=search_space,
                                 n_iter=200,
                                 cv=5,
                                 n_jobs=-1,
                                 verbose=1, scoring='f1')


# Fit the search instance
rand_search.fit(X_train, y_train)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 354 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done 704 tasks      | elapsed:   24.8s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   33.0s finished


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('prep',
                                              ColumnTransformer(transformers=[('conti',
                                                                               Pipeline(steps=[('sca',
                                                                                                StandardScaler()),
                                                                                               ('imputer',
                                                                                                SimpleImputer(strategy='median'))]),
                                                                               [1,
                                                                                4,
                                                                                5,
                                                                                6,
                                           

In [ ]:
print(rand_search.best_params_, rand_search.best_score_)

{'clf__n_estimators': 500, 'clf__min_samples_split': 6, 'clf__min_samples_leaf': 6, 'clf__max_leaf_nodes': 50, 'clf__max_features': 0.1, 'clf__max_depth': 9, 'clf__bootstrap': False, 'clf': RandomForestClassifier(bootstrap=False, max_depth=9, max_features=0.1,
                       max_leaf_nodes=50, min_samples_leaf=6,
                       min_samples_split=6, n_estimators=500, n_jobs=-1)} 0.8394431235053629


In [ ]:
# Baysian Optimization for Hyperparameters
# Caution!! Takes long time!
import optuna


def objective(trial):

    # Invoke suggest methods of a Trial object to generate hyperparameters.
    classifier_names = trial.suggest_categorical(
        'reg', ['pipe1', 'pipe2', 'pipe3', 'pipe4'])
    if classifier_names == 'pipe1':
        n_esti = trial.suggest_int('n_esti', 100, 1000)
        msl = trial.suggest_int('msl', 1, 20)
        md = trial.suggest_int('md', 1, 20)
        mf = trial.suggest_uniform('mf', 0.1, 0.5)
        mln = trial.suggest_int('mln', 10, 200)
        bs = trial.suggest_categorical('bs', [True, False])
        mss = trial.suggest_int('mss', 2, 20)
        cw = trial.suggest_categorical(
            'cw', ['balanced', 'balanced_subsample', None])

        pipee = Pipeline([('prep', prep),
                          ('clf', RandomForestClassifier(max_features=mf,
                                                         n_jobs=-1,
                                                         n_estimators=n_esti,
                                                         min_samples_leaf=msl,
                                                         max_depth=md,
                                                         class_weight=cw,
                                                         max_leaf_nodes=mln,
                                                         bootstrap=bs,
                                                         min_samples_split=mss
                                                         ))])

    elif classifier_names == 'pipe2':
        n_esti = trial.suggest_int('n_esti', 100, 1000)

        learning_rate = trial.suggest_uniform('lr', 0.1, 1)
        pipee = Pipeline([('prep', prep), ('clf', AdaBoostClassifier(
            n_estimators=n_esti,
            learning_rate=learning_rate
        ))])
    elif classifier_names == 'pipe3':
        msl = trial.suggest_int('msl', 1, 20)
        md = trial.suggest_int('md', 1, 20)
        mln = trial.suggest_int('mln', 10, 200)
        learning_rate = trial.suggest_uniform('lr', 0.1, 1)
        l2_regularization = trial.suggest_loguniform('l2', 0.00001, 10)
        pipee = Pipeline([('prep', prep),
                          ('clf', HistGradientBoostingClassifier(
                              min_samples_leaf=msl,
                              max_depth=md,
                              max_leaf_nodes=mln,
                              learning_rate=learning_rate,
                              l2_regularization=l2_regularization,
                              ))])
    else:
        n_esti = trial.suggest_int('n_esti', 100, 1000)
        msl = trial.suggest_int('msl', 1, 20)
        md = trial.suggest_int('md', 1, 20)
        mln = trial.suggest_int('mln', 10, 200)
        mss = trial.suggest_int('mss', 2, 20)
        learning_rate = trial.suggest_uniform('lr', 0.1, 1)
        pipee = Pipeline([('prep', prep),
                          ('clf', GradientBoostingClassifier(
                              n_estimators=n_esti,
                              min_samples_leaf=msl,
                              max_depth=md,
                              max_leaf_nodes=mln,
                              learning_rate=learning_rate,
                              min_samples_split=mss
                              ))])

    # choose f1 score as it is suitable for both imbalanced data and balanced data
    scores = cross_validate(pipee, X_train, y_train,
                            scoring='f1', cv=5, n_jobs=-1)

    error = -np.mean(scores['test_score'])

    return error  # An objective value linked with the Trial object.


study = optuna.create_study()  # Create a new study.
study.optimize(objective, n_trials=100, n_jobs=1, show_progress_bar=1)
print(study.best_params)
study.trials_dataframe().sort_values('value').head(10)

[I 2021-03-09 23:21:56,084] A new study created in memory with name: no-name-2e802082-f46a-4e73-959c-db75b402a3f0
/home/zlyu/anaconda3/lib/python3.8/site-packages/optuna/progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


[I 2021-03-09 23:21:59,776] Trial 0 finished with value: -0.7566590828476268 and parameters: {'reg': 'pipe4', 'n_esti': 948, 'msl': 1, 'md': 13, 'mln': 179, 'mss': 17, 'lr': 0.16282386367513363}. Best is trial 0 with value: -0.7566590828476268.
[I 2021-03-09 23:22:01,394] Trial 1 finished with value: -0.7450046861457169 and parameters: {'reg': 'pipe4', 'n_esti': 691, 'msl': 10, 'md': 16, 'mln': 28, 'mss': 6, 'lr': 0.5438630242165365}. Best is trial 0 with value: -0.7566590828476268.
[I 2021-03-09 23:22:01,512] Trial 2 finished with value: -0.8103499651640197 and parameters: {'reg': 'pipe3', 'msl': 13, 'md': 2, 'mln': 138, 'lr': 0.7970155110261475, 'l2': 1.1304784152803096}. Best is trial 2 with value: -0.8103499651640197.
[I 2021-03-09 23:22:03,193] Trial 3 finished with value: -0.8028961865574828 and parameters: {'reg': 'pipe2', 'n_esti': 999, 'lr': 0.5531559708372202}. Best is trial 2 with value: -0.8103499651640197.
[I 2021-03-09 23:22:04,147] Trial 4 finished with value: -0.8294207

,number,value,datetime_start,datetime_complete,duration,params_bs,params_cw,params_l2,params_lr,params_md,params_mf,params_mln,params_msl,params_mss,params_n_esti,params_reg,state
91,91,-0.842093,2021-03-09 23:23:07.767778,2021-03-09 23:23:08.123215,0 days 00:00:00.355437,True,None,NaN,NaN,5.0,0.113198,75.0,2.0,12.0,203.0,pipe1,COMPLETE
141,141,-0.841771,2021-03-09 23:23:28.487520,2021-03-09 23:23:28.809821,0 days 00:00:00.322301,False,None,NaN,NaN,11.0,0.105909,60.0,8.0,10.0,201.0,pipe1,COMPLETE
74,74,-0.840596,2021-03-09 23:22:47.085609,2021-03-09 23:22:48.435966,0 days 00:00:01.350357,True,None,NaN,NaN,6.0,0.100323,27.0,2.0,14.0,916.0,pipe1,COMPLETE
32,32,-0.840333,2021-03-09 23:22:21.759295,2021-03-09 23:22:22.190788,0 days 00:00:00.431493,False,None,NaN,NaN,11.0,0.113843,18.0,8.0,7.0,320.0,pipe1,COMPLETE
96,96,-0.839895,2021-03-09 23:23:10.785951,2021-03-09 23:23:11.247639,0 days 00:00:00.461688,True,None,NaN,NaN,7.0,0.115148,70.0,3.0,12.0,269.0,pipe1,COMPLETE
102,102,-0.839718,2021-03-09 23:23:14.385325,2021-03-09 23:23:14.764898,0 days 00:00:00.379573,True,None,NaN,NaN,5.0,0.100548,80.0,1.0,10.0,213.0,pipe1,COMPLETE
83,83,-0.839637,2021-03-09 23:22:57.853861,2021-03-09 23:22:59.246071,0 days 00:00:01.392210,True,None,NaN,NaN,3.0,0.115133,44.0,2.0,12.0,946.0,pipe1,COMPLETE
143,143,-0.839466,2021-03-09 23:23:29.173746,2021-03-09 23:23:29.552551,0 days 00:00:00.378805,False,None,NaN,NaN,11.0,0.115863,60.0,8.0,10.0,270.0,pipe1,COMPLETE
36,36,-0.839450,2021-03-09 23:22:23.179327,2021-03-09 23:22:23.404602,0 days 00:00:00.225275,False,None,NaN,NaN,10.0,0.103319,22.0,9.0,16.0,152.0,pipe1,COMPLETE
115,115,-0.839205,2021-03-09 23:23:19.566209,2021-03-09 23:23:19.864747,0 days 00:00:00.298538,False,None,NaN,NaN,7.0,0.144724,87.0,2.0,10.0,190.0,pipe1,COMPLETE


In [ ]:
# Collected best hyperparameters with some hand tuning
params0 = dict(bootstrap=False, max_depth=19, max_features=0.1,
               max_leaf_nodes=170, min_samples_split=10,
               n_estimators=300, n_jobs=-1)

params1 = {'n_estimators': 100,
           'learning_rate': 0.129}

params2 = {'min_samples_leaf': 10,
           'max_depth': 1,
           'max_leaf_nodes': 90,
           'learning_rate': 0.49978304475143825,
           'l2_regularization': 1.677161576446008e-05}

params3 = {'min_samples_leaf': 13,
           'max_depth': 2,
           'max_leaf_nodes': 45,
           'learning_rate': 0.27,
           'l2_regularization': 6.711291}

params4 = {'n_estimators': 130,
           'learning_rate': 0.161163}

params5 = {'n_estimators': 138,
           'min_samples_leaf': 12,
           'max_depth': 1,
           'max_leaf_nodes': 103,
           'min_samples_split': 19,
           'learning_rate': 0.5245992478750252}

params6 = {'n_estimators': 154,
           'min_samples_leaf': 11,
           'max_depth': 1,
           'max_leaf_nodes': 119,
           'min_samples_split': 10,
           'learning_rate': 0.5707539214903328}

params7 = {'n_estimators': 645,
           'min_samples_leaf': 2,
           'max_depth': 17,
           'max_features': 0.17,
           'max_leaf_nodes': 74,
           'bootstrap': False,
           'min_samples_split': 11,
           'class_weight': None}

In [ ]:
# Plus in hyperparameters into model pipelines

# Candidate from 1st Randomized search
pipe0 = Pipeline([('prep0', prep),
                  ('clf0',
                   RandomForestClassifier(**params0))])

# Candidate from 2nd Randomized search
pipe1 = Pipeline([('prep5', prep),
                  ('clf',
                   AdaBoostClassifier(**params1))])

# Candidate from 1st Bayesian Optimization search
pipe2 = Pipeline([('prep4', prep),
                  ('clf', HistGradientBoostingClassifier(**params2))])

# Candidate from 2nd Bayesian Optimization search

pipe3 = Pipeline([('prep4', prep),
                  ('clf',
                   HistGradientBoostingClassifier(**params3))])

# Candidate from 3rd Bayesian Optimization search

pipe4 = Pipeline([('prep5', prep),
                  ('clf',
                   AdaBoostClassifier(**params4))])

# Candidate from 4th Bayesian Optimization search
pipe5 = Pipeline([('prep6', prep),
                  ('clf',
                   GradientBoostingClassifier(**params5))])

# Candidate from 5th Bayesian Optimization search
pipe6 = Pipeline([('prep6', prep),
                  ('clf',
                   GradientBoostingClassifier(**params6))])

# Candidate from 6th Bayesian Optimization search
pipe7 = Pipeline([('prep', prep),
                  ('clf',
                   RandomForestClassifier(**params7))])

In [ ]:
# compare performance with iterative cross-validation
# to minimize effect of data split variance.
scores = []
for i in range(8):  # number of candidates
    score = []
    for j in range(11):  # number of iterations for each candidate
        score.append(np.mean(cross_validate(globals()['pipe'+str(i)],
                                            X_train, y_train, scoring='f1',
                                            cv=5, n_jobs=-1)['test_score']))
    scores.append((np.mean(score), np.std(score),
                   'pipe' +
                   str(i), type(
                       (globals()['pipe'+str(i)]).steps[1][1]).__name__[:-10],
                   (globals()['params'+str(i)])))

In [ ]:
ranking = pd.DataFrame(scores,
                       columns=['mean_score', 'std', 'estimator', 'type', 'param'])\
    .sort_values('mean_score', ascending=False)\
    .reset_index(drop=True)\
    .sort_values('type')

In [ ]:
pd.options.display.max_colwidth = 200
ranking

,mean_score,std,estimator,type,param
3,0.824813,0.000000e+00,pipe4,AdaBoost,"{'n_estimators': 130, 'learning_rate': 0.161163}"
7,0.821332,1.110223e-16,pipe1,AdaBoost,"{'n_estimators': 100, 'learning_rate': 0.129}"
5,0.823229,0.000000e+00,pipe6,GradientBoosting,"{'n_estimators': 154, 'min_samples_leaf': 11, 'max_depth': 1, 'max_leaf_nodes': 119, 'min_samples_split': 10, 'learning_rate': 0.5707539214903328}"
6,0.821701,0.000000e+00,pipe5,GradientBoosting,"{'n_estimators': 138, 'min_samples_leaf': 12, 'max_depth': 1, 'max_leaf_nodes': 103, 'min_samples_split': 19, 'learning_rate': 0.5245992478750252}"
2,0.830104,0.000000e+00,pipe3,HistGradientBoosting,"{'min_samples_leaf': 13, 'max_depth': 2, 'max_leaf_nodes': 45, 'learning_rate': 0.27, 'l2_regularization': 6.711291}"
4,0.824786,0.000000e+00,pipe2,HistGradientBoosting,"{'min_samples_leaf': 10, 'max_depth': 1, 'max_leaf_nodes': 90, 'learning_rate': 0.49978304475143825, 'l2_regularization': 1.677161576446008e-05}"
0,0.832748,3.381317e-03,pipe7,RandomForest,"{'n_estimators': 645, 'min_samples_leaf': 2, 'max_depth': 17, 'max_features': 0.17, 'max_leaf_nodes': 74, 'bootstrap': False, 'min_samples_split': 11, 'class_weight': None}"
1,0.831261,3.140749e-03,pipe0,RandomForest,"{'bootstrap': False, 'max_depth': 19, 'max_features': 0.1, 'max_leaf_nodes': 170, 'min_samples_split': 10, 'n_estimators': 300, 'n_jobs': -1}"


In [ ]:
# Automaticlly select best models
best_rf = globals()[ranking[ranking['type'] == 'RandomForest'].iloc[0, 2]]
best_ada = globals()[ranking[ranking['type'] == 'AdaBoost'].iloc[0, 2]]
best_gbm = globals()[ranking[ranking['type']
                             == 'GradientBoosting'].iloc[0, 2]]
best_hist = globals()[ranking[ranking['type']
                              == 'HistGradientBoosting'].iloc[0, 2]]

In [ ]:
# Best in each family:Random Forest, GBM, Hist GBM, AdaBoost
estimators = [('p0', best_rf), ('p1', best_ada),
              ('p2', best_gbm), ('p3', best_hist)]

In [ ]:
# stacked ensemble
stack = StackingClassifier(estimators, cv=5, n_jobs=-1)

In [ ]:
# voting ensemble
vote = VotingClassifier(estimators, n_jobs=-1)

In [ ]:
# evaluate stacking and voting with cross validation
scores = []
for ensemble in ['stack', 'vote']:
    score = []
    for i in range(5):
        score.append(np.mean(cross_validate(globals()[ensemble], X_train, y_train,
                                            scoring='f1', cv=5, n_jobs=-1)
                             ['test_score']))
    scores.append((np.mean(score), np.std(score),
                   ensemble, type((globals()[ensemble])).__name__[:-10],
                   f'Ensemble of best Random Forest, GBM, Hist GBM and AdaBoost models'))

In [ ]:
ranking2 = pd.DataFrame(scores,
                        columns=['mean_score', 'std', 'estimator', 'type', 'param'])

In [ ]:
ranking_final = pd.concat([ranking, ranking2], axis=0)\
.sort_values('mean_score', ascending=False)\
            .reset_index(drop=True)\

In [ ]:
ranking_final

,mean_score,std,estimator,type,param
0,0.832748,3.381317e-03,pipe7,RandomForest,"{'n_estimators': 645, 'min_samples_leaf': 2, 'max_depth': 17, 'max_features': 0.17, 'max_leaf_nodes': 74, 'bootstrap': False, 'min_samples_split': 11, 'class_weight': None}"
1,0.831261,3.140749e-03,pipe0,RandomForest,"{'bootstrap': False, 'max_depth': 19, 'max_features': 0.1, 'max_leaf_nodes': 170, 'min_samples_split': 10, 'n_estimators': 300, 'n_jobs': -1}"
2,0.830104,0.000000e+00,pipe3,HistGradientBoosting,"{'min_samples_leaf': 13, 'max_depth': 2, 'max_leaf_nodes': 45, 'learning_rate': 0.27, 'l2_regularization': 6.711291}"
3,0.827674,2.011502e-03,stack,Stacking,"Ensemble of best Random Forest, GBM, Hist GBM and AdaBoost models"
4,0.824813,0.000000e+00,pipe4,AdaBoost,"{'n_estimators': 130, 'learning_rate': 0.161163}"
5,0.824786,0.000000e+00,pipe2,HistGradientBoosting,"{'min_samples_leaf': 10, 'max_depth': 1, 'max_leaf_nodes': 90, 'learning_rate': 0.49978304475143825, 'l2_regularization': 1.677161576446008e-05}"
6,0.823229,0.000000e+00,pipe6,GradientBoosting,"{'n_estimators': 154, 'min_samples_leaf': 11, 'max_depth': 1, 'max_leaf_nodes': 119, 'min_samples_split': 10, 'learning_rate': 0.5707539214903328}"
7,0.822580,1.186932e-03,vote,Voting,"Ensemble of best Random Forest, GBM, Hist GBM and AdaBoost models"
8,0.821701,0.000000e+00,pipe5,GradientBoosting,"{'n_estimators': 138, 'min_samples_leaf': 12, 'max_depth': 1, 'max_leaf_nodes': 103, 'min_samples_split': 19, 'learning_rate': 0.5245992478750252}"
9,0.821332,1.110223e-16,pipe1,AdaBoost,"{'n_estimators': 100, 'learning_rate': 0.129}"


In [ ]:
best_model = globals()[ranking_final.loc[0, 'estimator']]

<b>According to final scores,we chose this model below as our final model:</b>





In [ ]:
if type(best_model).__name__ == 'Pipeline':
    print(f'Final model is a {type(best_model.steps[1][1]).__name__}')
else:
    print(f'Final model is a {type(best_model).__name__}')
print()
print(f'With preprocessing pipeline:\n {prep}')
print()
print(f"With hyperparameters:\n{ranking_final.loc[0,'param']}")

Final model is a RandomForestClassifier

With preprocessing pipeline:
 ColumnTransformer(transformers=[('conti',
                                 Pipeline(steps=[('sca', StandardScaler()),
                                                 ('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 [1, 4, 5, 6, 8, 9, 12]),
                                ('cate',
                                 Pipeline(steps=[('str',
                                                  FunctionTransformer(func=<function <lambda> at 0x7fd0b8185ee0>)),
                                                 ('imputer',
                                                  SimpleImputer(fill_value='unknown',
                                                                strategy='constant')),
                                                 ('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
   

In [ ]:
best_model.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('conti',
                                                  Pipeline(steps=[('sca',
                                                                   StandardScaler()),
                                                                  ('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  [1, 4, 5, 6, 8, 9, 12]),
                                                 ('cate',
                                                  Pipeline(steps=[('str',
                                                                   FunctionTransformer(func=<function <lambda> at 0x7fd0b8185ee0>)),
                                                                  ('imputer',
                                                                   SimpleImputer(fill_value='unknown',
                                                     

### Final evaluation on test set


In [ ]:
y_pred = best_model.predict(X_test)

In [ ]:
f1score = f1_score(y_test, y_pred)
print(f'F1 score for prediction on test set is {f1score:.4f}')

F1 score for prediction on test set is 0.8561


In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy score for prediction on test set is {accuracy:.4f}')

Accuracy score for prediction on test set is 0.8304


In [ ]:
r = permutation_importance(
    best_model, X, y, scoring='f1', n_repeats=10, n_jobs=-1)

In [ ]:
for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{X.columns[i]:<8}"
              f"{r.importances_mean[i]:.3f}"
              f" ± {r.importances_std[i]:.3f}")

cp      0.067 ± 0.006
exang   0.045 ± 0.006
from    0.021 ± 0.002
age     0.018 ± 0.003
sex     0.018 ± 0.003
thalach 0.017 ± 0.002
thal    0.017 ± 0.003
slope   0.017 ± 0.003
chol    0.015 ± 0.004
restecg 0.010 ± 0.002
trestbps0.009 ± 0.002
ca      0.008 ± 0.002
fbs     0.004 ± 0.002


### Conclusion

Randomized hyperparameter search and bayesian optimization search are very useful to find decent hyperparameters quickly.

Tree ensembles are effective in tabular data prediction as it has plenty of mearsures to reduce overfit. And thet worked as expected in this project. Well-tuned Gradient Boost Machine works best in this case.

The final model is decent to help doctors to diagnose if a patient has heart disease or not.
And it shows that CP (chest pain type) and EXANG (exercise induced angina (1 = yes; 0 = no)) are the first 2 important factors in the diagnosis.

### Next steps

Other ensembles such as Xgboost, catboost and lightgbm could be attempted in the future.